# Statistical Analysis of Clustering Results
This notebook performs statistical analysis on the clustering results, including ANOVA for 3-cluster solutions and t-tests for 2-cluster solutions. The analysis is performed both with and without dummy variables to ensure robust results.

## 1. Setup and Data Loading

- Import required libraries
- Load clustering results
- Configure visualization settings

In [1]:
# Import required libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import f_oneway, ttest_ind
import numpy as np
import warnings
import os
from scipy import stats
warnings.filterwarnings('ignore')

# Set global visualization parameters
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12
sns.set_palette("Set2")

plt.ioff()  # Turn off interactive plotting

# Helper functions
def perform_statistical_analysis(df, n_clusters=3):
    """
    Performs statistical analysis for either 2 or 3 clusters
    Args:
        df: DataFrame with cluster assignments
        n_clusters: Number of clusters to analyze
    Returns:
        DataFrame of significant variables
    """
    if n_clusters == 3:
        return perform_anova_analysis(df)
    return perform_ttest_analysis(df)

def perform_anova_analysis(df):
    """
    Performs ANOVA analysis for 3 clusters
    Args:
        df: DataFrame with cluster assignments
    Returns:
        DataFrame with ANOVA results
    """
    groups = df['KMeans_Cluster'].unique()
    p_values = {}
    
    for column in df.columns:
        if column != 'KMeans_Cluster':
            group_data = [df[df['KMeans_Cluster'] == group][column] for group in groups]
            f_statistic, p_value = f_oneway(*group_data)
            p_values[column] = p_value
            
    return process_results(df, p_values)

def perform_ttest_analysis(df):
    """
    Performs t-test analysis for 2 clusters
    Args:
        df: DataFrame with cluster assignments
    Returns:
        DataFrame with t-test results
    """
    group_X = df[df['KMeans_Cluster'] == 0]
    group_Y = df[df['KMeans_Cluster'] == 1]
    
    results = []
    for col in df.columns:
        if col != 'KMeans_Cluster':
            t_stat, p_value = stats.ttest_ind(group_X[col], group_Y[col], equal_var=False)
            results.append((col, p_value))
            
    return process_results(df, dict(results))

def es_dummy(col):
    """
    Check if a column contains only binary values (0/1)
    Args:
        col: DataFrame column to check
    Returns:
        bool: True if column contains only binary values
    """
    return col.dropna().isin([0, 1]).all()

label_mapping = {'roi4': 'Negative ROI',
 'roi_yes': 'ROI Measured',
 'ecp_extredeployment': 'External Redeployment',
 'p_effect_reverse': 'Program Effectiveness',
 'evp_network': 'Cross-departmental Networks',
 'roi5': 'Positive ROI',
 'sum_tr_sk': 'Sum of Trained Skills',
 'sha_b_sk_n_digital': 'Share Needed Digital Skills',
 'stat_government': 'Help Org Use Gov Subsidy',
 'ecp_intredeployment': 'Internal Redeployment',
 'stat_csr': 'Fulfilling CSR Requirement',
 'reason_dei': 'Reason DEI',
 'sk_selected': 'Total Nr skills needed',
 'stand2': 'Mix Standardization customization',
 'f_union_1 - 25%': 'Union Share 1-25%',
 'inc_mgr_nofin': 'Manager: Non-financial Incentive',
 'invest_cont': 'Continued Investment',
 'p_fund_gov': 'Funded by Government',
 'f_medium': 'Medium Firm Size (100-999)',
 'roi2': 'Not yet but intend to calculate ROI',
 'roi3': 'Tried to but unable to',
 'roi1': 'No attempt to calculate',
 'p_eligibility': 'Participation Eligibility',
 'p_targetfunc_it_Not_Selected': 'Target Function: IT not selected',
 'p_cont_investment_Very likely': 'Cont. Investment Very Likely',
 'p_adv_hr': 'Advocate HR',
 'p_participated_2023_1000 - 9999': 'Participated in 2023 (1000 - 9999)',
 'p_program_length': 'Program Length (Years)',
 'p_fund_org': 'Funded by Org',
 'p_criteria_jobtitle': 'Selection Criteria: Job Title',
 'p_criteria_assmskills': 'Selection Criteria: Assessment of Skills',
 'p_target_emp': 'Target Group: Employees',
 'p_challenge_progcompl': 'Challenge: Program Completion',
 'p_year_end_2023': 'Program End: 2023',
 'p_part': 'Number of participants',
 'p_fund_wrk': 'Funded by Worker',
 'sha_b_sk_n_soft': 'Needed share soft skill',
 'sha_b_sk_n_man': 'Needed share mgmt skill',
 'p_part_exp': 'Expected Participation',
 'tot_kpi_tracked': 'Total Nr of KPIs',
 'f_union_1___25%': 'Union share 1-25%',
 'p_targetfunc_it': 'Target function IT',
 'size': 'Q1_2: How many people does your organization employ?',
 'm_tenure_firm': 'Tenure of Respondent',
 'noanswerQ5_27_n1': 'noanswerQ5_27_n1: How was this upskilling program funded?: I don’t know - No Ans',
 'noanswerQ7_29_n1': 'noanswerQ7_29_n1: How was this reskilling program funded?: I don’t know - No Ans',
 'f_naics': 'Firm: 2 digits NAICS industry',
 'HP_region4': 'HP_region4: Hidden Question. Region4',
 'HP_region9': 'HP_region9: Hidden Question. Region9',
 'f_size': 'RECODE of f_size_n',
 'f_geography': 'Firm: Where do majority of employees work',
 'f_geography_grouped': 'Geography - grouped',
 'f_hq': 'Firm: HQ location',
 'f_mne': 'Firm: MNE dummy',
 'f_sub_number': 'Firm: Number of continents firm is active in',
 'f_export': 'Firm: Dummy if firm exports',
 'f_own': 'Firm: Ownership',
 'f_naics_clone': 'Firm: 2 digits NAICS industry',
 'f_naics_super': 'Superindustry',
 'f_naics_super2': 'Superindustry 2',
 'f_union': 'Firm: Union %',
 'f_public': 'Firm: Dummy publicly listed',
 'f_subsidy': 'Firm: Utilized a subsidy for initiatives',
 'program': 'Program type',
 'm_role': 'Role of Respondent',
 'hr_workforce': 'D: Engagement in Workforce activity',
 'hr_sk_description': 'D: Skill taxonomy needed to perform in given occupation',
 'hr_sk_inventory_use': 'D: How useful is the inventory of skills for workforce planning activities?',
 'hr_academy': 'D: Investment in setting up academy/school',
 'hr_internalmkt': 'D: advertised to specific employees and no permission needed',
 'p_participated': 'Program: Number of people who have participated since inception',
 'p_participated_2023': 'Program: Number of people who will have participated end of 2023',
 'p_mandavolunt': 'Was the program mandatory or opt-in for majority?',
 'p_year_start': 'Program Start Year',
 'p_year_end': 'Program End Year',
 'p_hourstrained': 'Total hours trained (per employee)',
 'p_duration': 'Duration of program',
 'p_comphours': 'Program during working hours',
 'p_otjactivities': 'On-the-job activities included',
 'p_cost': 'Estimated cost per person',
 'p_adequatefund': 'Was the program adequately funded?',
 'p_adequatefunddummy': 'Adequately funded Dummy',
 'p_fund_union': 'Union',
 'p_fund_other': 'Other',
 'p_advocacy': 'Who is the primary advocate for investments in the program?',
 'p_responsibility': 'Who has the primary responsibility to implement the program?',
 'p_application': 'Who could apply?',
 'p_selection': 'Were participants selected by org?',
 'p_criteria_tenure': 'Criteria - Tenure',
 'p_criteria_qualifications': 'Criteria - Pre-existing skills/qualifications',
 'p_criteria_assmsmotivation': 'Criteria - Pre-assessment of motivation',
 'p_criteria_managerrec': "Criteria - Manager's recommendation'",
 'p_criteria_other': 'Criteria - Other',
 'p_targetemp_c': 'Target - C-suite',
 'p_targetemp_bul': 'Target - Unit Leaders',
 'p_targetemp_mm': 'Target - Middle Managers',
 'p_targetemp_emp': 'Target - Employees',
 'p_targetfunc_leg': 'Target - Legal',
 'p_targetfunc_hr': 'Target - HR',
 'p_targetfunc_adm': 'Target - Administration',
 'p_targetfunc_op': 'Target - Operations/Manufacturing',
 'p_targetfunc_mrksal': 'Target - Marketing/Sales',
 'p_targetfunc_rd': 'Target - Research/Development',
 'p_targetfunc_accfin': 'Target - Accounting/Finance',
 'p_targetfunc_cust': 'Target - Customer Service',
 'p_difloc': 'Was the program rolled out in different locations',
 'p_difstand': 'Was the program standardized across locations',
 'p_challenge_convincingemployees': 'Challenge - Convincing eligible employees to apply',
 'p_challenge_selecting': 'Challenge - Selecting the right participants',
 'p_challenge_learning': 'Challenge - Ensuring that participants learned new skills',
 'p_challenge_convmanager': 'Challenge - Convincing managers to allow their employees to participate in the p',
 'p_challenge_linkprogramcareer': 'Challenge - Linking the program to career progressions',
 'p_challenge_effectiveness': 'Challenge - Monitoring the effectiveness of the program',
 'p_challenge_scaling': 'Challenge - Scaling-up the program',
 'p_challenge_determreturn': 'Challenge - Determining financial returns of programs relative to other policies',
 'p_challenge_supportoutsideHR': 'Challenge - Getting the support of other corporate managers outside HR',
 'p_challenge_funding': 'Challenge - Finding adequate resources to fund the program',
 'p_challenge_other': 'Challenge - Other',
 'p_cont_investment': 'Liklihood of continued investment in program',
 'dd_pilot_length': 'Length of pilot/experimental phase (or none)',
 'dd_pilot_dummy': 'Pilot Dummy variable',
 'dd_pilot_ct': 'Did the pilot have control/treatment?',
 'dd_pilot_over': 'Is experiment/pilot over?',
 'dd_design_board': 'Design - Board of Directors',
 'dd_design_c': 'Design - C-Suite Leaders',
 'dd_design_hr': 'Design - HR Leaders',
 'dd_design_bul': 'Design - Unit/Function Leaders',
 'dd_design_mm': 'Design - Middle Managers',
 'dd_design_emp': 'Design - Employees',
 'dd_design_union': 'Design - Unions',
 'dd_design_ia': 'Design - Internal Academy',
 'dd_external': 'Were external parties involved?',
 'dd_extdesign_ta': 'External Designer - Trade Association',
 'dd_extdesign_gov': 'External Designer - Government',
 'dd_extdesign_acad': 'External Designer - Academic Institutions',
 'dd_extdesign_lsshelf': 'External Designer - Learning Specialist (Off-the-Shelf)',
 'dd_extdesign_custom': 'External Designer - Learning Specialist (Custom)',
 'dd_extdesign_all': 'External Designers - Combined Categories',
 'dd_delivery': 'Delivery - Who delivered the program?',
 'mot_mgr': 'Difficulty to get managers to support',
 'mot_wrk': 'Difficulty to get employees to enroll',
 'mot_mgragg': 'Difficulty to get managers to support',
 'mot_wrkagg': 'Difficulty to get employees to enroll',
 'inc_mgr_fin': 'Manager Incentive - Financial',
 'inc_mgr_rec': 'Manager Incentive - Other Form of Recognition',
 'inc_mgr_intrin': 'Manager Incentive - Intrinsic',
 'inc_mgr_none': 'Manager Incentive - No Incentive',
 'inc_mgr_all': 'Manager Incentive - Combined Incentive Categories',
 'inc_wrk_career': 'Worker Incentive - Informed of Career Benefit',
 'inc_wrk_risk': 'Worker Incentive - Informed of Risk',
 'inc_wrk_fin': 'Worker Incentive - Financial Incentive',
 'inc_wrk_nofin': 'Worker Incentive - Non-Financial Incentive',
 'inc_wrk_cert': 'Worker Incentive - Certificate or Badge',
 'inc_wrk_rec': 'Worker Incentive - Other Form of Recognition',
 'inc_wrk_strat': 'Worker Incentive - Informed of Strategic Importance',
 'inc_wrk_job': 'Worker Incentive - Job Offer',
 'inc_wrk_all': 'Worker Incentive - Combined Categories',
 'exp_participation': 'Were expectation about participation met?',
 'p_finassessment': 'Final assessment to assess whether skills improved',
 'k_track_numberppl': 'How many ppl started program',
 'k_track_hours': 'Total hours completed',
 'k_track_attract': 'Whether program attracked right participants',
 'k_track_completion': 'How many people completed program',
 'k_track_changeskill': 'Whether participants saw change in skill set',
 'k_track_changecareer': 'Change in career trajectory/moved to different role',
 'k_track_otherHR': 'Other HR related outcomes (e.g., reduced attrition)',
 'k_track_busoutcomes': 'Business Outcomes',
 'k_track_benefitstraining': 'Benefits vs. alternative approaches',
 'k_track_other': 'Other',
 'k_track_none': 'We did not track any of these metrics',
 'k_review_BoD': 'Review - Board of Directors',
 'k_review_c': 'Review - C-Suite Leaders',
 'k_review_hr': 'Review - HR Leaders',
 'k_review_bul': 'Review - Unit/Function Leaders',
 'k_review_mm': 'Review - Middle Managers',
 'k_review_emp': 'Review - Employees',
 'k_review_union': 'Review - Unions',
 'k_review_ia': 'Review - Internal Academy',
 'k_review_other': 'Review - Other',
 'k_reviewcombined_c': 'Review - Board of Directors + C-Suite',
 'k_reviewcombined_hr': 'Review - HR Leaders',
 'k_reviewcombined_others': 'Review - Others',
 'k_freq': 'How often were the KPIs reviewed?',
 'p_effectiveness': 'Subjective Effectiveness of Program',
 'p_roi': 'ROI achieved',
 'dum_sk_gap1': 'sk_gap_overall==     0.0000',
 'dum_sk_gap2': 'sk_gap_overall==     1.0000',
 'dum_sk_gap3': 'sk_gap_overall==     2.0000',
 'clus4': 'Cluster ID',
 'cc1': 'clus4==     1.0000',
 'cc2': 'clus4==     2.0000',
 'cc3': 'clus4==     3.0000',
 'cc4': 'clus4==     4.0000',
 'clusplot': 'Cluster ID',
 'alt_cc1': 'clus5==     1.0000',
 'alt_cc2': 'clus5==     2.0000',
 'alt_cc3': 'clus5==     3.0000',
 'alt_cc4': 'clus5==     4.0000',
 'alt_cc5': 'clus5==     5.0000',
 'sk_gapcertainty': 'Skill gap: Certainty about gap',
 'tr_sk_n_soft_leader': 'Leadership Skills - Included in Training',
 'tr_sk_n_soft_adapt': 'Adaptability and Flexibility - Included in Training',
 'tr_sk_n_man_comm': 'Communication Skills - Included in Training',
 'tr_sk_n_man_problemsolving': 'Problem-Solving Skills - Included in Training',
 'tr_sk_n_man_projectman': 'Project Management Skills - Included in Training',
 'tr_sk_n_f_industry': 'Industry-Specific Knowledge - Included in Training',
 'tr_sk_n_f_customer': 'Customer Service Skills - Included in Training',
 'tr_sk_n_f_marketing': 'Sales and Marketing Skills - Included in Training',
 'tr_sk_n_f_finance': 'Financial Management Skills - Included in Training',
 'tr_sk_n_f_hr': 'Human Resources Skills - Included in Training',
 'tr_sk_n_f_supplychain': 'Supply Chain and Logistics Skills - Included in Training',
 'tr_sk_n_dig_basic': 'Basic Digital Skills - Included in Training',
 'tr_sk_n_dig_advance': 'Advanced Digital Skills - Included in Training',
 'tr_sk_n_dig_prod': 'Production-Related Digital Skills - Included in Training',
 'tr_sk_n_dig_data': 'Data Analysis and Processing Skills - Included in Training',
 'tr_sk_n_career': 'Career Transition Skills - Included in Training',
 'tr_sk_n_other': 'Other Skills - Included in Training',
 'tr_clus4': 'Cluster ID',
 'dd_clus1': 'tr_clusplot2==Digital+Cognitive',
 'dd_clus2': 'tr_clusplot2==Everything',
 'dd_clus3': 'tr_clusplot2==Soft',
 'dd_clus4': 'tr_clusplot2==Digital+Soft',
 'ee_clus1': 'tr_clusplot2==Digital+Cognitive',
 'ee_clus2': 'tr_clusplot2==Everything',
 'ee_clus3': 'tr_clusplot2==Soft',
 'ee_clus4': 'tr_clusplot2==Digital+Soft',
 'reason_tech': 'New Technology',
 'reason_prodinn': 'New Products',
 'reason_bmodel': 'New Business Model',
 'reason_sust': 'New Sustainability Objectives',
 'reason_other': 'Other',
 'evp_attract': 'Attracting Talent',
 'evp_engage': 'Keeping Employees Engaged',
 'evp_retain': 'Retaining Employees',
 'evp_orgculture': 'Contributing to Learning Org Culture',
 'evp_diversity': 'Building a More Diverse Org',
 'reasongroup_evp': 'EVP Employee',
 'reasongroup_evpculture': 'EVP Culture',
 'ecp_promotion': 'Help Qualify for Internal Promotion Opportunities',
 'ecp_other': 'Other',
 'stat_dev_noexist': 'Help Develop Skills that Do not Exist in External LaborMrkt',
 'stat_dev_expensive': 'Help Develop Skills that are Too Expensive to Hire/Contract',
 'p_size_coarse1': 'p_participated_coarse==     1.0000',
 'p_size_coarse2': 'p_participated_coarse==     2.0000',
 'p_size_coarse3': 'p_participated_coarse==     3.0000',
 'pdur1': 'p_duration==< 1 month',
 'pdur2': 'p_duration==2 - 6 months',
 'pdur3': 'p_duration==7 - 12 months',
 'pdur4': 'p_duration== > 12 months',
 'phours1': 'p_hourstrained==< 10 hours',
 'phours2': 'p_hourstrained==10 - 49 hours',
 'phours3': 'p_hourstrained==50 - 99 hours',
 'phours4': 'p_hourstrained==100 - 199 hours',
 'phours5': 'p_hourstrained==200 - 499 hours',
 'phours6': 'p_hourstrained==> 500 hours',
 'pcomp1': 'p_comphours==Entirely compensated hours',
 'pcomp2': 'p_comphours==Partly compensated hours',
 'pcomp3': 'p_comphours==Uncompensated hours',
 'padv1': 'p_advocacy_hier==Top Management',
 'padv2': 'p_advocacy_hier==HR',
 'padv3': 'p_advocacy_hier==BU and Middle Management',
 'padv4': 'p_advocacy_hier==Employees and Others',
 'presp1': 'p_responsibility_hier==Top Management',
 'presp2': 'p_responsibility_hier==HR',
 'presp3': 'p_responsibility_hier==BU and Middle Management',
 'presp4': 'p_responsibility_hier==Employees and Others',
 'pdelivery1': 'dd_delivery==External Instructors only',
 'pdelivery2': 'dd_delivery==External and Internal',
 'pdelivery3': 'dd_delivery==Internal Instructers only',
 'pmand1': 'p_mandavolunt==Mandatory',
 'pmand2': 'p_mandavolunt==Voluntary',
 'papp1': 'p_application==Anyone could apply regardless of their function/department',
 'papp2': 'p_application==Anyone could apply if belonging to specific function/department/h',
 'papp3': 'p_application==Only people nominated by managers',
 'papp4': 'p_application==Other',
 'psel1': 'p_selection==Yes',
 'psel2': 'p_selection==No',
 'inc_man1': 'inc_mgr_all==No Incentive',
 'inc_man2': 'inc_mgr_all==Non-financial Incentive',
 'inc_man3': 'inc_mgr_all==Financial Incentive',
 'inc_emp1': 'inc_wrk_all==Non-financial Incentive',
 'inc_emp2': 'inc_wrk_all==Badge or Certificate',
 'inc_emp3': 'inc_wrk_all==Financial Incentive',
 'inc_emp4': 'inc_wrk_all==Job Offer',
 'exppart1': 'exp_participation==Exceeded',
 'exppart2': 'exp_participation==Met',
 'exppart3': 'exp_participation==Below',
 'pcost1': 'p_cost==< $500',
 'pcost2': 'p_cost==$501 - $1,000',
 'pcost3': 'p_cost==$1,001 - $5,000',
 'pcost4': 'p_cost==$5,001 - $10,000',
 'pcost5': 'p_cost==> $10,000',
 'stand1': 'p_difstand==Very standard across geographies',
 'stand3': 'p_difstand==Very flexible',
 'pilot1': 'dd_pilot_length==No pilot',
 'pilot2': 'dd_pilot_length==Less than 3 months',
 'pilot3': 'dd_pilot_length==Between 3 months and 1 year',
 'pilot4': 'dd_pilot_length==More than 1 year',
 'control1': 'dd_pilot_ct==Yes',
 'control2': 'dd_pilot_ct==No',
 'assess1': 'p_finassessment==Yes',
 'assess2': 'p_finassessment==No',
 'kpiinput1': 'kpi_input==     0.0000',
 'kpiinput2': 'kpi_input==     1.0000',
 'kpioutput_micro1': 'kpi_output_micro==     0.0000',
 'kpioutput_micro2': 'kpi_output_micro==     1.0000',
 'kpioutput_macro1': 'kpi_output_macro==     0.0000',
 'kpioutput_macro2': 'kpi_output_macro==     1.0000'
 }


def create_statistical_table(df, group_col, results, n_groups):
    """Create statistical table with means and standard deviations"""
    groups = df[group_col].unique()
    
    # Create LaTeX table
    if n_groups == 3:
        latex_table = "\\begin{tabular}{lcccccc}\n"
        latex_table += "Feature & Mean G1 & SD G1 & Mean G2 & SD G2 & Mean G3 & SD G3 & p-value \\\\\n"
    else:
        latex_table = "\\begin{tabular}{lcccc}\n"
        latex_table += "Feature & Mean G1 & SD G1 & Mean G2 & SD G2 & p-value \\\\\n"
    
    latex_table += "\\hline\n"
    
    # Create pandas DataFrame for visualization
    table_data = []
    columns = ['Feature']
    for i in range(n_groups):
        columns.extend([f'Mean G{i+1}', f'SD G{i+1}'])
    columns.append('p-value')
    
    for variable, p_value in results:
        variable_label = label_mapping.get(variable, variable)
        means = []
        sds = []
        # row_data = [variable]
        row_data = [variable_label]
        for group in range(n_groups):
            group_data = df[df[group_col] == group][variable]
            mean = group_data.mean()
            sd = group_data.std()
            means.append(mean)
            sds.append(sd)
            row_data.extend([f"{mean:.2f}", f"{sd:.2f}"])
        
        # Add to LaTeX table
        stats_str = " & ".join([f"{m:.2f} & {s:.2f}" for m, s in zip(means, sds)])
        #latex_table += f"{variable} & {stats_str} & {p_value:.4f} \\\\\n"
        latex_table += f"{variable_label} & {stats_str} & {p_value:.4f} \\\\\n"
        
        # Add to pandas data
        row_data.append(f"{p_value:.4f}")
        table_data.append(row_data)
    
    latex_table += "\\end{tabular}"
    
    # Create pandas DataFrame
    pd_table = pd.DataFrame(table_data, columns=columns)
    
    return latex_table, pd_table

def plot_density_distributions(df, variables, group_col, title_prefix="", n_clusters=None, analysis_type="all_vars"):
    """
    Create and save density plots for specified variables
    Args:
        df: DataFrame containing the data
        variables: List of variables to plot
        group_col: Column name for cluster assignments
        title_prefix: Prefix for plot titles
        n_clusters: Number of clusters (2 or 3)
        analysis_type: Type of analysis ("all_vars" or "no_dummies")
    """
    # Create base output directory if it doesn't exist
    base_dir = "../Output/Figures"
    cluster_dir = f"{base_dir}/{n_clusters}_clusters"
    analysis_dir = f"{cluster_dir}/{analysis_type}"
    
    # Create nested directories if they don't exist
    for dir_path in [base_dir, cluster_dir, analysis_dir]:
        os.makedirs(dir_path, exist_ok=True)
    
    for var in variables:
        plt.figure(figsize=(10, 6))
        for group in sorted(df[group_col].unique()):
            group_data = df[df[group_col] == group][var]
            sns.kdeplot(data=group_data, label=f'Cluster {group}', fill=True)
        
        plt.title(f'{title_prefix} {var}')
        plt.xlabel(var)
        plt.ylabel('Density')
        plt.legend()
        plt.tight_layout()
        
        # Save plot
        safe_var_name = "".join(x for x in var if x.isalnum() or x in ['-', '_'])
        plt.savefig(f"{analysis_dir}/density_{safe_var_name}.png")
        plt.close()
        
def process_results(df, p_values, alpha=0.05):
    """
    Processes statistical results and generates visualizations
    Args:
        df: DataFrame with data
        p_values: Dictionary of p-values by variable
        alpha: Significance level
    Returns:
        DataFrame of top significant variables
    """
    p_values_df = pd.DataFrame(list(p_values.items()), columns=['Variable', 'p_value'])
    significant_vars = p_values_df[p_values_df['p_value'] < alpha]
    top_vars = significant_vars.sort_values('p_value').head(20)
    
    create_density_plots(df, top_vars['Variable'].tolist())
    return top_vars

def create_density_plots(df, variables, n_clusters=None, analysis_type="all_vars"):
    """
    Creates density plots for significant variables
    Args:
        df: DataFrame containing the data
        variables: List of variables to plot
        n_clusters: Number of clusters (2 or 3)
        analysis_type: Type of analysis ("all_vars" or "no_dummies")
    """
    # Create base output directory if it doesn't exist
    base_dir = "../Output/Figures"
    cluster_dir = f"{base_dir}/{n_clusters}_clusters"
    analysis_dir = f"{cluster_dir}/{analysis_type}"
    
    # Create nested directories if they don't exist
    for dir_path in [base_dir, cluster_dir, analysis_dir]:
        os.makedirs(dir_path, exist_ok=True)
    
    for var in variables:
        plt.figure(figsize=(10, 6))
        g = sns.FacetGrid(df, hue='KMeans_Cluster', height=6, aspect=1.5)
        g.map(sns.kdeplot, var, fill=True)
        g.add_legend()
        plt.title(f'Density Plot for {var}')
        plt.tight_layout()
        
        # Save plot
        safe_var_name = "".join(x for x in var if x.isalnum() or x in ['-', '_'])
        plt.savefig(f"{analysis_dir}/density_facet_{safe_var_name}.png")
        plt.close()

## 2. Three-Cluster Analysis

- ANOVA testing for all variables
- Generation of statistical tables
- Density plots for significant variables
- Analysis without dummy variables

In [2]:
def analyze_clusters(df, n_clusters):
    """
    Main analysis function for clusters
    Args:
        df: DataFrame with cluster assignments
        n_clusters: Number of clusters to analyze
    """
    # Drop unnecessary columns if they exist
    if n_clusters == 3:
        df.drop(["tr_part", "tr_eligibility", "tr_part_exp"], axis=1, inplace=True)
    
    # Perform statistical analysis
    top_vars = perform_statistical_analysis(df, n_clusters)
    
    # Create plots and save them
    create_density_plots(df, top_vars['Variable'].tolist(), 
                        n_clusters=n_clusters, 
                        analysis_type="all_vars")
    
    # Generate and print LaTeX table and pandas table
    latex_table, pd_table = create_statistical_table(df, 'KMeans_Cluster', 
                                                   zip(top_vars['Variable'], top_vars['p_value']), 
                                                   n_clusters)
    print(f"\nLaTeX Table for {n_clusters}-Cluster Analysis:")
    print(latex_table)
    print(f"\nVisualized Table for {n_clusters}-Cluster Analysis:")
    display(pd_table)
    
    return top_vars

# Execute analysis for 3 clusters
df_3clusters = pd.read_csv("../Output/Results/clusters_3_construction.csv")
top_vars_3clusters = analyze_clusters(df_3clusters, 3)


LaTeX Table for 3-Cluster Analysis:
\begin{tabular}{lcccccc}
Feature & Mean G1 & SD G1 & Mean G2 & SD G2 & Mean G3 & SD G3 & p-value \\
\hline
Number of participants & 38.93 & 16.63 & 76.59 & 9.12 & 74.02 & 10.99 & 0.0000 \\
Participation Eligibility & 43.77 & 13.06 & 74.36 & 10.24 & 74.80 & 14.18 & 0.0000 \\
Expected Participation & 52.67 & 17.49 & 83.01 & 11.17 & 83.46 & 9.87 & 0.0000 \\
Funded by Org & 57.48 & 30.59 & 23.63 & 6.68 & 56.30 & 18.62 & 0.0000 \\
Funded by Government & 14.25 & 12.24 & 23.10 & 10.23 & 12.40 & 9.44 & 0.0000 \\
Financial Management Skills - Included in Training & 0.23 & 0.43 & 0.11 & 0.32 & 0.54 & 0.50 & 0.0000 \\
Other & 3.25 & 8.47 & 10.19 & 10.79 & 2.10 & 5.45 & 0.0000 \\
Funded by Worker & 12.33 & 12.11 & 22.87 & 10.91 & 15.20 & 10.97 & 0.0000 \\
p_responsibility_hier==HR & 0.32 & 0.47 & 0.74 & 0.44 & 0.64 & 0.48 & 0.0000 \\
p_resp_hr & 0.32 & 0.47 & 0.74 & 0.44 & 0.64 & 0.48 & 0.0000 \\
p_responsibility_HR Leaders & 0.32 & 0.47 & 0.74 & 0.44 & 0.64 & 

,Feature,Mean G1,SD G1,Mean G2,SD G2,Mean G3,SD G3,p-value
0,Number of participants,38.93,16.63,76.59,9.12,74.02,10.99,0.0000
1,Participation Eligibility,43.77,13.06,74.36,10.24,74.80,14.18,0.0000
2,Expected Participation,52.67,17.49,83.01,11.17,83.46,9.87,0.0000
3,Funded by Org,57.48,30.59,23.63,6.68,56.30,18.62,0.0000
4,Funded by Government,14.25,12.24,23.10,10.23,12.40,9.44,0.0000
5,Financial Management Skills - Included in Trai...,0.23,0.43,0.11,0.32,0.54,0.50,0.0000
6,Other,3.25,8.47,10.19,10.79,2.10,5.45,0.0000
7,Funded by Worker,12.33,12.11,22.87,10.91,15.20,10.97,0.0000
8,p_responsibility_hier==HR,0.32,0.47,0.74,0.44,0.64,0.48,0.0000
9,p_resp_hr,0.32,0.47,0.74,0.44,0.64,0.48,0.0000


<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

...

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

In [3]:
import umap
from sklearn.cluster import KMeans

def analyze_without_dummies(df, n_clusters):
    """
    Analysis excluding dummy variables
    Args:
        df: Original DataFrame
        n_clusters: Number of clusters to analyze
    """
    # Prepare data
    df_numeric = df.apply(pd.to_numeric, errors='coerce')
    df_numeric = df_numeric.dropna(axis=1, how='all')
    df_no_dummies = df_numeric.loc[:, ~df_numeric.apply(es_dummy)]
    df_no_year = df_no_dummies.loc[:, ~df_no_dummies.columns.str.contains('year', case=False)]
    
    # Perform clustering
    reducer = umap.UMAP(random_state=42)
    embedding = reducer.fit_transform(df_no_year.dropna())
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    
    # Prepare final dataset
    df_analysis = df_no_year.dropna().copy()
    df_analysis['KMeans_Cluster'] = kmeans.fit_predict(embedding)
    
    # Analyze and create visualizations
    top_vars = analyze_clusters(df_analysis, n_clusters)
    create_density_plots(df_analysis, top_vars['Variable'].tolist(),
                        n_clusters=n_clusters,
                        analysis_type="no_dummies")
    
    return df_analysis, top_vars


# Execute analysis without dummies for 3 clusters
df = pd.read_stata("../Data/V1_qualflags_analysis2_ML.dta")
df_analysis_3, top_vars_3 = analyze_without_dummies(df, 3)


LaTeX Table for 3-Cluster Analysis:
\begin{tabular}{lcccccc}
Feature & Mean G1 & SD G1 & Mean G2 & SD G2 & Mean G3 & SD G3 & p-value \\
\hline
Funded by Government & 21.61 & 14.97 & 0.96 & 3.66 & 21.70 & 11.87 & 0.0000 \\
Union & 15.99 & 13.20 & 0.42 & 2.97 & 19.22 & 8.59 & 0.0000 \\
Funded by Worker & 16.82 & 15.87 & 1.18 & 4.72 & 20.96 & 12.80 & 0.0000 \\
Funded by Org & 41.41 & 20.64 & 97.23 & 7.01 & 28.92 & 13.86 & 0.0000 \\
Expected Participation & 58.71 & 18.00 & 54.93 & 25.39 & 84.07 & 9.81 & 0.0000 \\
Number of participants & 48.99 & 18.37 & 41.43 & 23.72 & 76.22 & 11.31 & 0.0000 \\
Participation Eligibility & 54.23 & 17.49 & 54.18 & 23.38 & 74.04 & 12.65 & 0.0000 \\
Other & 4.16 & 8.53 & 0.21 & 1.82 & 9.21 & 10.32 & 0.0000 \\
p_participated_coarse & 1.96 & 0.77 & 1.76 & 0.80 & 2.30 & 0.77 & 0.0000 \\
Total Nr of KPIs & 3.51 & 1.82 & 4.00 & 1.85 & 3.33 & 1.42 & 0.0000 \\
Program Length (Years) & 1.67 & 1.12 & 1.91 & 1.18 & 1.49 & 1.03 & 0.0001 \\
f_size_n & 4.39 & 1.65 & 4.49 

,Feature,Mean G1,SD G1,Mean G2,SD G2,Mean G3,SD G3,p-value
0,Funded by Government,21.61,14.97,0.96,3.66,21.70,11.87,0.0000
1,Union,15.99,13.20,0.42,2.97,19.22,8.59,0.0000
2,Funded by Worker,16.82,15.87,1.18,4.72,20.96,12.80,0.0000
3,Funded by Org,41.41,20.64,97.23,7.01,28.92,13.86,0.0000
4,Expected Participation,58.71,18.00,54.93,25.39,84.07,9.81,0.0000
5,Number of participants,48.99,18.37,41.43,23.72,76.22,11.31,0.0000
6,Participation Eligibility,54.23,17.49,54.18,23.38,74.04,12.65,0.0000
7,Other,4.16,8.53,0.21,1.82,9.21,10.32,0.0000
8,p_participated_coarse,1.96,0.77,1.76,0.80,2.30,0.77,0.0000
9,Total Nr of KPIs,3.51,1.82,4.00,1.85,3.33,1.42,0.0000


<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

...

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

## 3. Two-Cluster Analysis

- T-test analysis for all variables
- Statistical tables generation
- Density plots for significant differences
- Analysis without dummy variables

In [4]:
# Execute analysis for 2 clusters
df_2clusters = pd.read_csv("../Output/Results/clusters_2_construction.csv")
top_vars_2clusters = analyze_clusters(df_2clusters, 2)
create_density_plots(df_2clusters, top_vars_2clusters['Variable'].tolist())

# Execute analysis without dummies for 2 clusters
df = pd.read_stata("../Data/V1_qualflags_analysis2_ML.dta")
df_analysis_2, top_vars_2 = analyze_without_dummies(df, 2)


LaTeX Table for 2-Cluster Analysis:
\begin{tabular}{lcccc}
Feature & Mean G1 & SD G1 & Mean G2 & SD G2 & p-value \\
\hline
Number of participants & 45.95 & 19.66 & 77.22 & 8.83 & 0.0000 \\
tr_part & 45.95 & 19.66 & 77.22 & 8.83 & 0.0000 \\
tr_eligibility & 50.96 & 17.83 & 74.94 & 12.05 & 0.0000 \\
Participation Eligibility & 50.96 & 17.83 & 74.94 & 12.05 & 0.0000 \\
Expected Participation & 59.84 & 20.18 & 83.57 & 10.78 & 0.0000 \\
tr_part_exp & 59.84 & 20.18 & 83.57 & 10.78 & 0.0000 \\
Funded by Org & 61.30 & 28.42 & 30.14 & 12.66 & 0.0000 \\
Union & 10.95 & 12.98 & 20.00 & 8.67 & 0.0000 \\
Funded by Worker & 12.12 & 12.28 & 21.31 & 10.53 & 0.0000 \\
p_resp_hr & 0.39 & 0.49 & 0.72 & 0.45 & 0.0000 \\
p_responsibility_hier==HR & 0.39 & 0.49 & 0.72 & 0.45 & 0.0000 \\
fsub & 0.66 & 0.48 & 0.93 & 0.26 & 0.0000 \\
Funded by Government & 13.06 & 11.87 & 20.47 & 10.53 & 0.0000 \\
Other & 2.56 & 7.49 & 8.08 & 10.22 & 0.0000 \\
p_ongoing & 0.55 & 0.50 & 0.27 & 0.45 & 0.0001 \\
p_target_top & 0

,Feature,Mean G1,SD G1,Mean G2,SD G2,p-value
0,Number of participants,45.95,19.66,77.22,8.83,0.0000
1,tr_part,45.95,19.66,77.22,8.83,0.0000
2,tr_eligibility,50.96,17.83,74.94,12.05,0.0000
3,Participation Eligibility,50.96,17.83,74.94,12.05,0.0000
4,Expected Participation,59.84,20.18,83.57,10.78,0.0000
5,tr_part_exp,59.84,20.18,83.57,10.78,0.0000
6,Funded by Org,61.30,28.42,30.14,12.66,0.0000
7,Union,10.95,12.98,20.00,8.67,0.0000
8,Funded by Worker,12.12,12.28,21.31,10.53,0.0000
9,p_resp_hr,0.39,0.49,0.72,0.45,0.0000



LaTeX Table for 2-Cluster Analysis:
\begin{tabular}{lcccc}
Feature & Mean G1 & SD G1 & Mean G2 & SD G2 & p-value \\
\hline
Funded by Org & 33.43 & 17.20 & 91.39 & 18.16 & 0.0000 \\
Union & 18.77 & 10.48 & 1.00 & 5.21 & 0.0000 \\
Funded by Worker & 19.82 & 13.69 & 2.17 & 8.67 & 0.0000 \\
Number of participants & 66.33 & 16.79 & 38.68 & 23.31 & 0.0000 \\
tr_part & 66.33 & 16.79 & 38.68 & 23.31 & 0.0000 \\
Funded by Government & 20.95 & 12.54 & 4.86 & 12.72 & 0.0000 \\
tr_part_exp & 74.68 & 15.87 & 52.21 & 25.25 & 0.0000 \\
Expected Participation & 74.68 & 15.87 & 52.21 & 25.25 & 0.0000 \\
Other & 7.03 & 9.84 & 0.58 & 3.63 & 0.0000 \\
Participation Eligibility & 66.99 & 15.85 & 51.23 & 23.34 & 0.0000 \\
tr_eligibility & 66.99 & 15.85 & 51.23 & 23.34 & 0.0000 \\
p_participated_coarse & 2.17 & 0.77 & 1.75 & 0.80 & 0.0000 \\
Total Nr of KPIs & 3.30 & 1.51 & 4.10 & 1.93 & 0.0000 \\
Program Effectiveness & 2.57 & 0.51 & 2.42 & 0.51 & 0.0000 \\
Program Length (Years) & 1.58 & 1.07 & 1.86 & 1.1

,Feature,Mean G1,SD G1,Mean G2,SD G2,p-value
0,Funded by Org,33.43,17.20,91.39,18.16,0.0000
1,Union,18.77,10.48,1.00,5.21,0.0000
2,Funded by Worker,19.82,13.69,2.17,8.67,0.0000
3,Number of participants,66.33,16.79,38.68,23.31,0.0000
4,tr_part,66.33,16.79,38.68,23.31,0.0000
5,Funded by Government,20.95,12.54,4.86,12.72,0.0000
6,tr_part_exp,74.68,15.87,52.21,25.25,0.0000
7,Expected Participation,74.68,15.87,52.21,25.25,0.0000
8,Other,7.03,9.84,0.58,3.63,0.0000
9,Participation Eligibility,66.99,15.85,51.23,23.34,0.0000


<Figure size 1000x600 with 0 Axes>

...

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

## 4. Cluster and Program Type Comparison  

- UMAP visualization comparing clusters and actual program types  
- Confusion matrix showing program type distribution within clusters  
- Normalized proportions for each cluster  
- Evaluation of cluster homogeneity  

### Files Generated  

- `umap_cluster_program_comparison.png`: UMAP visualization  
- `cluster_program_distribution.csv`: Distribution statistics  
- `cluster_program_distribution.png`: Bar chart visualization  


In [7]:
from sklearn.preprocessing import StandardScaler

def clean_and_preprocess(data):
    """
    Clean and preprocess data with thorough NaN handling
    """
    # Create copy to avoid modifying original data
    df = data.copy()
    
    # Step 1: Create dummy variables
    df = pd.get_dummies(df)
    
    # Step 2: Clean column names
    df.columns = [col.replace('>', 'greater').replace('<', 'less').replace(',', '_') 
                 for col in df.columns]
    
    # Step 3: Handle missing values
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    
    # Fill numeric columns with median
    for col in numeric_cols:
        df[col] = df[col].fillna(df[col].median())
    
    # Fill non-numeric columns with mode
    non_numeric_cols = df.select_dtypes(exclude=['float64', 'int64']).columns
    for col in non_numeric_cols:
        df[col] = df[col].fillna(df[col].mode()[0] if not df[col].mode().empty else 0)
    
    # Step 4: Verify no NaN values remain
    if df.isna().any().any():
        print("Warning: NaN values still present after cleaning")
        # Drop any remaining rows with NaN values
        df = df.dropna()
    
    # Step 5: Convert all to float64 for scaling
    df = df.astype(float)
    
    return df

def compare_clusters_with_program_types(data_path="../Data/V1_qualflags_analysis2_ML.dta", 
                                      output_dir="../Output/Figures/"):
    """
    Create UMAP visualization comparing clusters with actual program types
    """
    # Load data
    print("Loading data...")
    data = pd.read_stata(data_path)
    program_types = data['program'].copy()  # Save original program types
    
    # Print initial data info
    print("\nInitial data shape:", data.shape)
    print("Missing values before preprocessing:", data.isna().sum().sum())
    
    # Preprocess data
    print("\nPreprocessing data...")
    data_cleaned = clean_and_preprocess(data)
    
    print("Data shape after preprocessing:", data_cleaned.shape)
    print("Missing values after preprocessing:", data_cleaned.isna().sum().sum())
    
    # Standardize features
    print("\nStandardizing features...")
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data_cleaned)
    
    # Verify scaled data
    if np.isnan(scaled_data).any():
        raise ValueError("NaN values found in scaled data")
    
    # Perform UMAP
    print("\nPerforming UMAP...")
    reducer = umap.UMAP(random_state=42)
    embedding = reducer.fit_transform(scaled_data)
    
    # Create visualizations
    print("\nCreating visualizations...")
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    
    # Plot 1: K-means clusters
    kmeans = KMeans(n_clusters=2, random_state=42)
    cluster_labels = kmeans.fit_predict(embedding)
    
    scatter1 = ax1.scatter(embedding[:, 0], embedding[:, 1], 
                          c=cluster_labels, cmap='viridis',
                          alpha=0.6)
    ax1.set_title('UMAP Projection with K-means Clusters (k=2)')
    ax1.set_xlabel('UMAP1')
    ax1.set_ylabel('UMAP2')
    plt.colorbar(scatter1, ax=ax1, label='Cluster')
    
    # Plot 2: Program types
    program_colors = {'Reskilling': '#FF6B6B', 'Upskilling': '#4ECDC4', 'General': '#95A5A6'}
    
    # Ensure program_types length matches embedding length
    program_types = program_types.iloc[data_cleaned.index]
    
    for program in program_colors:
        mask = program_types == program
        ax2.scatter(embedding[mask, 0], embedding[mask, 1], 
                   c=program_colors[program], label=program,
                   alpha=0.6)
    
    ax2.set_title('UMAP Projection by Program Type')
    ax2.set_xlabel('UMAP1')
    ax2.set_ylabel('UMAP2')
    ax2.legend(title='Program Type')
    
    plt.tight_layout()
    
    # Save figure
    print("\nSaving outputs...")
    plt.savefig(f"{output_dir}umap_cluster_program_comparison.png", 
                dpi=300, bbox_inches='tight')
    plt.close()
    
    # Create confusion matrix
    cluster_program_df = pd.DataFrame({
        'Cluster': cluster_labels,
        'Program': program_types.values
    })
    
    confusion_matrix = pd.crosstab(
        cluster_program_df['Cluster'],
        cluster_program_df['Program'],
        normalize='index'
    )
    
    # Save confusion matrix
    confusion_matrix.to_csv(f"{output_dir}cluster_program_distribution.csv")
    
    # Distribution plot
    plt.figure(figsize=(10, 6))
    confusion_matrix.plot(kind='bar', stacked=True)
    plt.title('Distribution of Program Types within Clusters')
    plt.xlabel('Cluster')
    plt.ylabel('Proportion')
    plt.legend(title='Program Type')
    plt.tight_layout()
    plt.savefig(f"{output_dir}cluster_program_distribution.png",
                dpi=300, bbox_inches='tight')
    plt.close()
    
    return confusion_matrix

# Run analysis
confusion_matrix = compare_clusters_with_program_types()
print("\nDistribution of program types within clusters:")
print(confusion_matrix)

Loading data...

Initial data shape: (1209, 466)
Missing values before preprocessing: 48704

Preprocessing data...
Data shape after preprocessing: (1209, 760)
Missing values after preprocessing: 0

Standardizing features...

Performing UMAP...

Creating visualizations...

Saving outputs...

Distribution of program types within clusters:
Program   General  Upskilling  Reskilling
Cluster                                  
0        0.135084    0.397749    0.467167
1        0.000000    0.372781    0.627219


<Figure size 1000x600 with 0 Axes>